# Networking (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/argu/6.1.5/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/Runtime.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let inline testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 120
|> _equal (Some false)

00:00:00 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let inline testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 120 65535
|> Async.RunSynchronously
|> _equal false

00:00:00 #2 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
False


## waitForPortAccess

In [ ]:
let inline waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Verbose (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5555

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    use listener = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 2 else 2)
            (if Runtime.isWindows () then 5 else 1500)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 80 else 80)
            (if Runtime.isWindows () then 150 else 600)

        true
    | _ -> false
|> _equal true

00:00:00 #3 [Debug] 1
00:00:00 #4 [Debug] _1
00:00:00 #5 [Debug] 2
00:00:00 #6 [Debug] 3
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #8 [Verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:00 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #10 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #11 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #12 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #13 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #14 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #15 [Ve

In [ ]:
//// test

let port = 5555

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    use listener = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 4 else 2)
            (if Runtime.isWindows () then 15 else 150)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 5 else 0)
            (if Runtime.isWindows () then 20 else 60)

        true
    | _ -> false
|> _equal true

00:00:08 #438 [Debug] 1
00:00:08 #439 [Debug] 2
00:00:08 #440 [Debug] _1
00:00:08 #441 [Debug] 3
00:00:08 #442 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #443 [Verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:08 #444 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #445 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #446 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #447 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #448 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:08 #449 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refuse

## getAvailablePort

In [ ]:
let inline getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5555

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    use listener1 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    use listener2 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 240
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 1200)

        true
    | _ -> false
|> _equal true

00:00:09 #492 [Debug] 1
00:00:09 #493 [Debug] 2
00:00:09 #494 [Debug] _1
00:00:09 #495 [Debug] _2
00:00:09 #496 [Debug] _3
00:00:09 #497 [Debug] 3
00:00:09 #498 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:09 #499 [Debug] 4
00:00:09 #500 [Verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:10 #501 [Verbose] waitForPortAccess / port: 5555 / retry: 100
00:00:12 #502 [Verbose] waitForPortAccess / port: 5555 / retry: 200
00:00:13 #503 [Verbose] waitForPortAccess / port: 5555 / retry: 300
00:00:13 #504 [Debug] _4
00:00:13 #505 [Debug] _5
00:00:13 #506 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:13 #507 [Debug] 5
00:00:13 #508 [Debug] 6
5557
313
313
313
True


In [ ]:
//// test

let port = 5555

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    use listener1 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    use listener2 = new System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 240
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 2 else 1)
            (if Runtime.isWindows () then 10 else 120)

        true
    | _ -> false
|> _equal true

00:00:13 #509 [Debug] 1
00:00:13 #511 [Debug] 2
00:00:13 #510 [Debug] _1
00:00:13 #512 [Debug] _2
00:00:13 #513 [Debug] _3
00:00:14 #514 [Debug] 3
00:00:14 #515 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:14 #516 [Debug] 4
00:00:14 #517 [Verbose] waitForPortAccess / port: 5555 / retry: 0
00:00:14 #518 [Debug] _4
00:00:14 #519 [Debug] _5
00:00:14 #520 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:14 #521 [Debug] 5
00:00:14 #522 [Debug] 6
5557
13
13
13
True
